In [524]:
# from sympy.assumptions.assume import global_assumptions
# from sympy.assumptions.ask import Q
# global_assumptions.add(Q.real(x))

In [525]:
import numpy as np
from sympy import symbols, cos, sin, Matrix, atan2, acos, sqrt, pi, Array
import sympy
import mpmath

# Helper functions

In [526]:
def A_dh(theta, d, a, alpha, type, radians = False):
    if not radians and not isinstance(theta, sympy.core.symbol.Symbol):
        theta = theta / 180 * sympy.pi
    if not radians and not isinstance(alpha, sympy.core.symbol.Symbol):
        alpha = alpha / 180 * sympy.pi
    return Matrix([
        [cos(theta), -sin(theta)*cos(alpha), sin(theta)*sin(alpha), a*cos(theta)],
        [sin(theta), cos(theta)*cos(alpha), -cos(theta)*sin(alpha), a*sin(theta)],
        [0, sin(alpha), cos(alpha), d],
        [0, 0, 0, 1]
    ])

def A_trans(x, y, z, type):
    return Matrix([
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])
    



# Config
 

In [527]:
theta_1, theta_2, theta_3, theta_4= symbols('theta_1 theta_2 theta_3 theta_4')

In [528]:
dh_conv = {
    1 : {'theta': theta_1, 'd': 50/1000, 'a': 0, 'alpha': 90, 'type': 'revolute'},
    2 : {'theta': theta_2, 'd': 0, 'a': 93/1000, 'alpha': 0, 'type': 'revolute'},
    3 : {'theta': theta_3, 'd': 0, 'a': 93/1000, 'alpha': 0, 'type': 'revolute'},
    4 : {'theta': theta_4, 'd': 0, 'a': 50/1000, 'alpha': 0, 'type': 'revolute'},
    5: {'x' : -15/1000, 'y' : 45/1000, 'z' : 0, 'type': 'transformation'}
}



In [529]:
for i, a in dh_conv.items():
    print(a)

{'theta': theta_1, 'd': 0.05, 'a': 0, 'alpha': 90, 'type': 'revolute'}
{'theta': theta_2, 'd': 0, 'a': 0.093, 'alpha': 0, 'type': 'revolute'}
{'theta': theta_3, 'd': 0, 'a': 0.093, 'alpha': 0, 'type': 'revolute'}
{'theta': theta_4, 'd': 0, 'a': 0.05, 'alpha': 0, 'type': 'revolute'}
{'x': -0.015, 'y': 0.045, 'z': 0, 'type': 'transformation'}


In [530]:
A = {
    '01' : A_dh(**dh_conv[1]),
    '12' : A_dh(**dh_conv[2]),
    '23' : A_dh(**dh_conv[3]),
    '34' : A_dh(**dh_conv[4]),
    '45' : A_trans(**dh_conv[5])
}

In [531]:
T = {
    '00' : sympy.eye(4),
    '01' : sympy.simplify(A['01']),
    '02' : sympy.simplify(A['01']*A['12']),
    '03' : sympy.simplify(A['01']*A['12']*A['23']),
    '04' : sympy.simplify(A['01']*A['12']*A['23']*A['34']),
    '05' : sympy.simplify(A['01']*A['12']*A['23']*A['34']*A['45'])
    # '05' : sympy.simplify(A['45']*A['34']*A['23']*A['12']*A['01'])
}

In [532]:
def jacobian_inner(t, joint, o_n):
    joint_type = joint['type']
    z = t[:3,2]
    o = t[:3,3]
    
    if joint_type == 'revolute':
        j = z.cross(o_n - o)
        return Matrix.vstack(j, z)
    else:
        return Matrix.vstack(z, Matrix.zeros(3,1))
    

def jacobian(T_, joints):
    o_n = T_[-1][:3,3]
    J = Matrix.zeros(6, len(joints))
    for idx, joint in enumerate(joints):
        t = T_[idx]
        j = jacobian_inner(t, joint, o_n)
        
        J[:, idx] = j
        
    return sympy.simplify(J)

def jacobian_all(T_, dh_conv_):
    J = {}
    for idx, joint in enumerate(dh_conv_):
        t = list(T_.values())[:idx+1]
        joints = list(dh_conv_.values())[:idx+1]
        J[idx] = jacobian(t, joints)
    return J

In [533]:
J_end = jacobian(list(T.values())[:4], list(dh_conv.values())[:3])
J_end


Matrix([
[-0.093*(cos(theta_2) + cos(theta_2 + theta_3))*sin(theta_1), (-0.093*sin(theta_2) - 0.093*sin(theta_2 + theta_3) + 6.93889390390723e-18)*cos(theta_1), (6.93889390390723e-18 - 0.093*sin(theta_2 + theta_3))*cos(theta_1)],
[ 0.093*(cos(theta_2) + cos(theta_2 + theta_3))*cos(theta_1), (-0.093*sin(theta_2) - 0.093*sin(theta_2 + theta_3) + 6.93889390390723e-18)*sin(theta_1), (6.93889390390723e-18 - 0.093*sin(theta_2 + theta_3))*sin(theta_1)],
[                                                          0,                                        0.093*cos(theta_2) + 0.093*cos(theta_2 + theta_3),                                       0.093*cos(theta_2 + theta_3)],
[                                                          0,                                                                             sin(theta_1),                                                       sin(theta_1)],
[                                                          0,                                              

In [534]:
def point_on_circle(phi: float, radius: float = 32/1000):
    centre = Array([150/1000, 0, 120/1000])
    return centre + radius * Array([0, cos(phi), sin(phi)])

In [540]:
def inv_kinematics(x,y,z,phi, dh_conv):
    """
    Hardcoded inverse kinematics for the robot arm
    """
    
    theta1 = atan2(y, x)
    
    a2 = dh_conv[2]['a']
    a3 = dh_conv[3]['a']
    a4 = dh_conv[4]['a']
    
    d1 = dh_conv[1]['d']
    
    r = sqrt(x**2 + y**2)
    r2 = cos(phi)*a4
    
    #Given r = r1 + r2
    r1 = r - r2
    
    z1 = d1
    z3 = sin(phi)*a4
    
    #Given z = z1 + z2 + z3
    z2 = z - z1 - z3
    
    c23 = sqrt(z2**2 + r1**2)
    
    c3 = ((c23**2) - (a2**2) - (a3**2)) / (2*a2*a3)
    
    
    D = (c23**2 - a2**2 - a3**2)/(-2*a2*a3)
    theta3 = -(atan2(sqrt(1-D**2), D) - pi)

    
    # #two solutions for theta3
    # print("c3", c3)
    
    # theta3_1 = atan2(-sqrt(1 - c3**2), c3)
    # theta3_2 = atan2(sqrt(1 - c3**2), c3)
    
    
    # print("theta3_1", theta3_1)
    # print("theta3_2", theta3_2)
    
    # theta3 = theta3_1
    
    theta2 = atan2(z2, r1) - atan2(a3*sin(theta3), a2 + a3*cos(theta3))
    
    theta4 = phi - theta2 - theta3
    
    
    return theta1, theta2, theta3, theta4

inv_kinematics(-0.032,0.15, 0.12, 0, dh_conv)


(-1.36061374013273 + pi,
 -0.239730216951784,
 -1.47169003299079 + pi,
 1.71142024994257 - pi)

In [541]:

def jacobian_point(J, _theta_1, _theta_2, _theta_3, _theta_4):
    """Computes jacobian given joint angles"""
    res = sympy.lambdify((theta_1, theta_2, theta_3, theta_4), J, 'sympy')
    return res(_theta_1, _theta_2, _theta_3, _theta_4)

def jacobian_circle_angle(J, phi, end_angle, dh_conv):
    """Computes jacobian given an angle of a circle"""
    
    point_on_circle(J, phi) #Compute coordinates of point on circle
    
    #Compute angles of joints
    _theta_1, _theta_2, _theta_3, _theta_4 = inv_kinematics(*point_on_circle(phi), end_angle, dh_conv)
    

    #Compute jacobian
    return jacobian_point(J, _theta_1, _theta_2, _theta_3, _theta_4)


In [542]:
J_end = jacobian(list(T.values())[:4], list(dh_conv.values())[:3])
J_end

# r1 = jacobian_circle_angle(J_end, 0)
# r2 = jacobian_circle_angle(J_end, pi/2)
r3 = jacobian_circle_angle(J_end, pi, 0, dh_conv)
# r4 = jacobian_circle_angle(J_end, 3*pi / 2)

# print(np.array(r1))
# print(np.array(r2))
# print((r3))
# print(np.array(r4))
r3

Matrix([
[0.0215680764114741, -0.0684594985497015, -0.0900555097184086],
[ 0.101100358178785,  0.0146046930239363,  0.0192118420732605],
[                 0,   0.103375356560303,  0.0130349642695982],
[                 0,  -0.208638471770519,  -0.208638471770519],
[                 0,  -0.977992836424308,  -0.977992836424308],
[                 1,                   0,                   0]])

In [549]:
J_cam = jacobian(list(T.values())[:5], list(dh_conv.values())[:4])
J_cam

ROUNDING_FACTOR = 3

r1 = jacobian_circle_angle(J_cam, 0, 0, dh_conv)
r2 = jacobian_circle_angle(J_cam, pi/2, 0, dh_conv)
r3 = jacobian_circle_angle(J_cam, pi, 0, dh_conv)
r4 = jacobian_circle_angle(J_cam, 3*pi / 2, 0, dh_conv)

print(np.round(np.array(r1).astype(np.float64), ROUNDING_FACTOR))
print(np.round(np.array(r2).astype(np.float64), ROUNDING_FACTOR))
print(np.round(np.array(r3).astype(np.float64), ROUNDING_FACTOR))
print(np.round(np.array(r4).astype(np.float64), ROUNDING_FACTOR))

[[-0.032 -0.068 -0.09   0.   ]
 [ 0.15  -0.015 -0.019  0.   ]
 [ 0.     0.153  0.063  0.05 ]
 [ 0.     0.209  0.209  0.209]
 [ 0.    -0.978 -0.978 -0.978]
 [ 1.     0.     0.     0.   ]]
[[ 0.    -0.102 -0.093  0.   ]
 [ 0.15   0.     0.     0.   ]
 [ 0.     0.15   0.057  0.05 ]
 [ 0.     0.     0.     0.   ]
 [ 0.    -1.    -1.    -1.   ]
 [ 1.     0.     0.     0.   ]]
[[ 0.032 -0.068 -0.09   0.   ]
 [ 0.15   0.015  0.019  0.   ]
 [ 0.     0.153  0.063  0.05 ]
 [ 0.    -0.209 -0.209 -0.209]
 [ 0.    -0.978 -0.978 -0.978]
 [ 1.     0.     0.     0.   ]]
[[ 0.    -0.038 -0.09   0.   ]
 [ 0.15   0.     0.     0.   ]
 [ 0.     0.15   0.073  0.05 ]
 [ 0.     0.     0.     0.   ]
 [ 0.    -1.    -1.    -1.   ]
 [ 1.     0.     0.     0.   ]]


In [550]:
J_cam = jacobian(list(T.values())[:6], list(dh_conv.values())[:5])
J_cam

r1 = jacobian_circle_angle(J_cam, 0, 0, dh_conv)
r2 = jacobian_circle_angle(J_cam, pi/2, 0, dh_conv)
r3 = jacobian_circle_angle(J_cam, pi, 0, dh_conv)
r4 = jacobian_circle_angle(J_cam, 3*pi / 2, 0, dh_conv)

print(np.round(np.array(r1).astype(np.float64), ROUNDING_FACTOR))
print(np.round(np.array(r2).astype(np.float64), ROUNDING_FACTOR))
print(np.round(np.array(r3).astype(np.float64), ROUNDING_FACTOR))
print(np.round(np.array(r4).astype(np.float64), ROUNDING_FACTOR))

[[-0.029 -0.112 -0.134 -0.044  0.209]
 [ 0.135 -0.024 -0.029 -0.009 -0.978]
 [ 0.     0.138  0.048  0.035  0.   ]
 [ 0.     0.209  0.209  0.209  0.   ]
 [ 0.    -0.978 -0.978 -0.978  0.   ]
 [ 1.     0.     0.     0.     0.   ]]
[[ 0.    -0.147 -0.138 -0.045  0.   ]
 [ 0.135  0.     0.     0.    -1.   ]
 [ 0.     0.135  0.042  0.035  0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.    -1.    -1.    -1.     0.   ]
 [ 1.     0.     0.     0.     0.   ]]
[[ 0.029 -0.112 -0.134 -0.044 -0.209]
 [ 0.135  0.024  0.029  0.009 -0.978]
 [ 0.     0.138  0.048  0.035  0.   ]
 [ 0.    -0.209 -0.209 -0.209  0.   ]
 [ 0.    -0.978 -0.978 -0.978  0.   ]
 [ 1.     0.     0.     0.     0.   ]]
[[ 0.    -0.083 -0.135 -0.045  0.   ]
 [ 0.135  0.     0.     0.    -1.   ]
 [ 0.     0.135  0.058  0.035  0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.    -1.    -1.    -1.     0.   ]
 [ 1.     0.     0.     0.     0.   ]]


In [553]:
T['01']

Matrix([
[cos(theta_1), 0,  sin(theta_1),    0],
[sin(theta_1), 0, -cos(theta_1),    0],
[           0, 1,             0, 0.05],
[           0, 0,             0,    1]])